In [1]:
!pip install datasets
!pip install translators

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 926.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
  Created wheel for PyExecJS: filename=PyExecJS-1.5.1-py3-none-any.whl size=14578 sha256=59e4a663bb8dd295e535212b73afbdd286171eefaf3da6ad19739cba8fff1dcd
  Stored in directory: /root/.cache/pip/wheels/9d/91/30/28e6da53d4f44dc445349b2ffad581968447e4cbc9dd7991b8
Successfully built PyExecJS


In [24]:
import numpy as np
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, BertForMultipleChoice,DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
import os
import sys
import logging
import torch



SP_train=np.load('/content/WP-train.npy',allow_pickle=True)
SP_test=np.load('/content/WP_eval_data_for_practice.npy',allow_pickle=True)
def deal(examples):
    data=pd.DataFrame()
    key=examples[0].keys()
    label=[]
    question=[]
    choice=[]
    for i in range(len(examples)):
        if 'label' in key:
            label.append(examples[i]['label'])
        question.append(examples[i]['question'])
        choice.append(examples[i]['choice_list'])
    data['question']=question
    A=[]
    B=[]
    C=[]
    D=[]
    for i in range(len(choice)):
        A.append(choice[i][0])
        B.append(choice[i][1])
        C.append(choice[i][2])
        D.append(choice[i][3])
    data['A']=A
    data['B']=B
    data['C']=C
    data['D']=D
    if label:
        data['label']=label
    return data



train_dict=deal(SP_train)
test_dict=deal(SP_test)
train=pd.DataFrame(train_dict)
test=pd.DataFrame(test_dict)

In [25]:
import translators as ts
import pandas as pd
from multiprocessing import Pool
from tqdm import tqdm

lang_dict = ['zh','ru','ar','fr','de','es','pt','it','ja']


# def translator_constructor(api):
#     if api == 'google':
#         return ts.google
#     elif api == 'bing':
#         return ts.bing
#     elif api == 'baidu':
#         return ts.baidu
#     elif api == 'sogou':
#         return ts.sogou
#     elif api == 'youdao':
#         return ts.youdao
#     elif api == 'tencent':
#         return ts.tencent
#     elif api == 'alibaba':
#         return ts.alibaba
#     else:
#         raise NotImplementedError(f'{api} translator is not realised!')

#尝试使用进程，和导师给的代码放在一起
def translate(data,language):
    question=[]
    choice_list=[]
    label=[]
    for i in range(len(data['question'])):
      res_question=ts.translate_text(data['question'][i],from_language='auto',to_language=language,translator='google')
      # res_A=ts.translate_text(data['A'][i],from_language='auto',to_language=language,translator='google')
      # res_B=ts.translate_text(data['B'][i],from_language='auto',to_language=language,translator='google')
      # res_C=ts.translate_text(data['C'][i],from_language='auto',to_language=language,translator='google')
      # res_D=ts.translate_text(data['D'][i],from_language='auto',to_language=language,translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      # res_A02=ts.translate_text(res_A,from_language='auto',to_language=language,translator='google')
      # res_B02=ts.translate_text(res_B,from_language='auto',to_language=language,translator='google')
      # res_C02=ts.translate_text(res_C,from_language='auto',to_language=language,translator='google')
      # res_D02=ts.translate_text(res_D,from_language='auto',to_language=language,translator='google')
      question.append(res_question02)
      #choice_list.append([res_A02,res_B02,res_C02,res_D02])
    data=pd.DataFrame()
    data['question']=question
    #data['choice_list']=choice_list
    data['label']=label
    return data

In [26]:
lang_dict = ['zh','ru','ar','fr','de','es','pt','it','ja']
#翻译成中文再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[0],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[0],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[0],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[0],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[0],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df

train_zh=pd.DataFrame()
train_zh=main()
#翻译成俄文再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[1],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[1],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[1],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[1],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[1],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_ru=pd.DataFrame()
train_ru=main()
#翻译成ar再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[2],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[2],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[2],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[2],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[2],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_ar=pd.DataFrame()
train_ar=main()

#翻译成fr再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[3],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[3],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[3],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[3],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[3],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_fr=pd.DataFrame()
train_fr=main()

#翻译成de再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[4],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[4],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[4],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[4],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[4],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_de=pd.DataFrame()
train_de=main()

#翻译成es再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[5],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[5],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[5],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[5],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[5],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_es=pd.DataFrame()
train_es=main()

#翻译成pt再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[6],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[6],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[6],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[6],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[6],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_pt=pd.DataFrame()
train_pt=main()

#翻译成it再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[7],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[7],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[7],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[7],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[7],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_it=pd.DataFrame()
train_it=main()

#翻译成ja再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[8],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[8],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[8],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[8],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[8],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02,x[5]]
    except:
        return [x[0],x[1],x[2],x[3],x[4],x[5]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D','label']] = imap_unordered_bar(translate,train[['question','A','B','C','D','label']].values)
    return df
train_ja=pd.DataFrame()
train_ja=main()

 24%|██▍       | 96/396 [00:30<00:09, 32.20it/s]
96it [00:30, 28.23it/s]
 37%|███▋      | 148/396 [00:57<01:58,  2.10it/s]
148it [00:57,  1.86it/s]
 47%|████▋     | 185/396 [00:59<00:08, 25.96it/s]
185it [00:59, 24.09it/s]
 99%|█████████▊| 391/396 [02:00<00:00,  6.26it/s]
396it [02:00,  3.28it/s]
 13%|█▎        | 50/396 [00:25<02:49,  2.04it/s]
48it [00:25, 36.52it/s]
 52%|█████▏    | 205/396 [01:07<00:45,  4.17it/s]
205it [01:07,  4.21it/s]
 56%|█████▌    | 222/396 [01:09<00:20,  8.45it/s]
222it [01:09,  8.07it/s]
 73%|███████▎  | 289/396 [01:34<01:59,  1.11s/it]
289it [01:34,  1.11s/it]
 81%|████████  | 319/396 [01:38<00:02, 28.96it/s]
319it [01:38, 24.60it/s]
 84%|████████▍ | 332/396 [01:39<00:03, 16.76it/s]
332it [01:39, 15.26it/s]
 86%|████████▌ | 341/396 [01:52<00:41,  1.31it/s]
341it [01:52,  1.06it/s]
 99%|█████████▉| 393/396 [01:56<00:00,  9.53it/s]
396it [01:56,  3.40it/s]
 10%|▉         | 39/396 [00:18<00:25, 13.85it/s]
39it [00:18, 13.93it/s]
 24%|██▎       | 94/396 [00:34<

In [27]:
lang_dict = ['zh','ru','ar','fr','de','es','pt','it','ja']
Data=pd.concat([train,train_zh,train_ru,train_ar,train_fr,train_de,train_es,train_pt,train_it,train_ja],axis=0)

In [28]:
Data.to_csv('/content/WP_train_translation.csv')

In [16]:
# 创建两个DataFrame
df1 = pd.DataFrame({'key':['a','b','c'],'value':[1,2,3]})
df2 = pd.DataFrame({'key':['d','e','f'],'value':[4,5,6]})

# 按行拼接
result = pd.concat([df1,df2],axis=0)
print(result)

  key  value
0   a      1
1   b      2
2   c      3
0   d      4
1   e      5
2   f      6


In [20]:
test

,question,A,B,C,D
0,"Everyone called him ""Batman,"" but he knew noth...",He tries to be friendly.,He is afraid others will laugh at him.,He was the star baseball player.,None of above.
1,"All of Mrs. Smith pets are dogs except one, an...",Mrs.Smith has one additional pet that is neith...,Mrs.Smith has one hybrid pet that is a dog and...,Mrs. Smith has one cat and one dog.,None of above.
2,Three doors are present. Every door has a way ...,"The third door, the alligator can be tierd and...",The second door with the lions. Since they hav...,"The first door, the first door is destroyed an...",None of above.
3,Tom and his younger sister were fighting. Thei...,"Tom's mother slid a newspaper under a door, ea...",Tom and his younger sister stand on the newspa...,The newspaper was folded multiple times and en...,None of above.
4,A woman sells a bottle of perfume to a man for...,Fifteen dollars.,Fifteen dollars plus the perfume.,Twenty-five dollars.,None of above.
...,...,...,...,...,...
115,How was it possible for a cowboy to arrive in ...,His horse is named Wednesday.,Friday and Saturday are holidays.,"While in town, he stays in bed for two days.",None of above.
116,Natives of the Arctic will never eat a penguin...,Penguins only live in Antarctica.,Penguins' eggs are toxic.,Penguins and their eggs are not part of their ...,None of above.
117,A full water glass was in the hands of the mag...,Surface tension prevents water from spilling.,The glass is too heavy.,"The glass was filled, but not with water.",None of above.
118,Which could see the best in complete darkness?...,Bat.,Tiger.,Owl.,None of above.


In [29]:
#翻译成中文再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[0],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[0],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[0],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[0],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[0],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df

test_zh=pd.DataFrame()
test_zh=main()
#翻译成俄文再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[1],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[1],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[1],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[1],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[1],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_ru=pd.DataFrame()
test_ru=main()
#翻译成ar再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[2],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[2],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[2],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[2],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[2],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_ar=pd.DataFrame()
test_ar=main()

#翻译成fr再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[3],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[3],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[3],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[3],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[3],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_fr=pd.DataFrame()
test_fr=main()

#翻译成de再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[4],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[4],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[4],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[4],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[4],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_de=pd.DataFrame()
test_de=main()

#翻译成es再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[5],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[5],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[5],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[5],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[5],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_es=pd.DataFrame()
test_es=main()

#翻译成pt再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[6],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[6],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[6],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[6],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[6],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_pt=pd.DataFrame()
test_pt=main()

#翻译成it再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[7],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[7],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[7],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[7],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[7],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,test[['question','A','B','C','D']].values)
    return df
test_it=pd.DataFrame()
test_it=main()

#翻译成ja再回译
def translate(x):
    try:
      res_question=ts.translate_text(x[0],from_language='auto',to_language=lang_dict[8],translator='google')
      res_A=ts.translate_text(x[1],from_language='auto',to_language=lang_dict[8],translator='google')
      res_B=ts.translate_text(x[2],from_language='auto',to_language=lang_dict[8],translator='google')
      res_C=ts.translate_text(x[3],from_language='auto',to_language=lang_dict[8],translator='google')
      res_D=ts.translate_text(x[4],from_language='auto',to_language=lang_dict[8],translator='google')
      res_question02=ts.translate_text(res_question,from_language='auto',to_language='en',translator='google')
      res_A02=ts.translate_text(res_A,from_language='auto',to_language='en',translator='google')
      res_B02=ts.translate_text(res_B,from_language='auto',to_language='en',translator='google')
      res_C02=ts.translate_text(res_C,from_language='auto',to_language='en',translator='google')
      res_D02=ts.translate_text(res_D,from_language='auto',to_language='en',translator='google')
      return [res_question02,res_A02,res_B02,res_C02,res_D02]
    except:
        return [x[0],x[1],x[2],x[3],x[4]]

def imap_unordered_bar(func, args, n_processes: int = 48):
    p = Pool(n_processes, maxtasksperchild=1000)
    res_list = []
    with tqdm(total=len(args)) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args))):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def main():
    df = pd.DataFrame()
    # tqdm.pandas('Translation progress')
    df[['question','A','B','C','D']] = imap_unordered_bar(translate,train[['question','A','B','C','D']].values)
    return df
test_ja=pd.DataFrame()
test_ja=main()

 18%|█▊        | 22/120 [00:13<00:14,  6.68it/s]
22it [00:13,  6.36it/s]
 52%|█████▏    | 62/120 [00:25<00:09,  5.98it/s]
62it [00:25,  5.89it/s]
 91%|█████████ | 109/120 [00:30<00:01,  6.81it/s]
120it [00:30,  3.93it/s]
 48%|████▊     | 57/120 [00:25<00:15,  4.06it/s]
57it [00:25,  4.06it/s]
 98%|█████████▊| 118/120 [00:32<00:00,  8.43it/s]
120it [00:32,  3.65it/s]
 40%|████      | 48/120 [00:16<00:01, 39.01it/s]
48it [00:16, 31.88it/s]
 57%|█████▋    | 68/120 [00:31<00:12,  4.24it/s]
68it [00:31,  4.70it/s]
 80%|████████  | 96/120 [00:32<00:01, 15.53it/s]
96it [00:32, 20.31it/s]
 88%|████████▊ | 106/120 [00:37<00:03,  4.59it/s]
120it [00:37,  3.20it/s]
 10%|█         | 12/120 [00:11<00:34,  3.13it/s]
12it [00:11,  3.10it/s]
 19%|█▉        | 23/120 [00:12<00:12,  7.51it/s]
24it [00:12,  8.68it/s]
 96%|█████████▌| 115/120 [00:30<00:00, 10.70it/s]
120it [00:30,  3.89it/s]
 38%|███▊      | 45/120 [00:14<00:05, 13.43it/s]
45it [00:14, 13.21it/s]
 97%|█████████▋| 116/120 [00:33<00:00,  6.4

In [30]:
Data_test=pd.concat([test,test_zh,test_ru,test_ar,test_fr,test_de,test_es,test_pt,test_it,test_ja],axis=0)
Data_test.to_csv('/content/WP_test_translation.csv')

In [31]:
Data_test

,question,A,B,C,D
0,What kind of nut has no shell?,A peanut.,A Doughnut.,A walnut.,None of above.
1,Which nut doesn't have a shell?,A Doughnut.,A walnut.,A peanut.,None of above.
2,Which type of bell doesn't make a sound?\n\n,A fire bell.,A cow bell.,A Bluebell.,None of above.
3,What does a stone become when in the water?,A whetstone.,A limestone,A sandstone.,None of above.
4,What changes a stone makes when submerged in w...,A whetstone.,A sandstone.,A limestone,None of above.
...,...,...,...,...,...
391,"What kind of ""stone"" can't be found in a quarry?",Cobblestone.,Sandstone.,Milestone.,None of above.
392,Which sort of bell doesn't ring?,A dumbbell.,A doorbell.,A cowbell.,None of above.
393,What kind of ice doesn't contain water?,Flaked ice.,Dry ice.,Glacier ice.,None of above.
394,What type of ice never melts?,Dice.,Flaked ice.,Glacier ice.,None of above.
